<a href="https://colab.research.google.com/github/lokkyX3/reddit-IMG-downloader/blob/main/reddit_IMG_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import praw
import requests
import time
from IPython.display import Image, display
from google.colab import files
from multiprocessing.pool import ThreadPool as Pool

In [ ]:
class ImageDownloader(object):

  counter = 0

  def __init__(self, target_user):
    # OAuth Credentials
    self.reddit = praw.Reddit(
                        client_id='',
                        client_secret='',
                        username='',
                        password='',
                        user_agent='',
                        )
    self.get_user_post(target_user)
  
  def download_images(self, sub_id, subreddit, url, counter):
    filename = f'{counter}_{sub_id}_{subreddit}.{url.split(".")[-1]}' #filename.file extension
    with open(filename,'wb') as img:
      img.write(requests.get(url, stream = True).content)
    files.download(filename)

  def download_imgur_album(self, sub_id, subreddit, url, counter):
    album = f'{counter}_{sub_id}_{subreddit}.{url.split("/")[-1]}' #filename.file extension
    with open(album,'wb') as img:
      img.write(requests.get(url, stream = True).content)
    files.download(album)

#Check whether submission is single image or an Imgur album 
  def process_function(self, submission):
    if submission.url.endswith(('.jpg', '.jpeg', '.png', '.gifs', '.gif', '.gifv')):
      self.download_images(submission.id, submission.subreddit.display_name, submission.url.replace('.gifv', '.mp4'), self.counter)
      self.counter+=1
    elif 'imgur.com/a' in submission.url:
      self.download_imgur_album(submission.id, submission.subreddit.display_name, submission.url+'/zip', self.counter)
      self.counter+=1

  def get_user_post(self, user):
    # for submission in self.reddit.redditor(user).submissions.top('all'):
    with Pool(2) as p:
      p.map(self.process_function,self.reddit.redditor(user).submissions.top('all'))




In [ ]:
start_time = time.time()
ImageDownloader('insert_username_here')
print(time.time()-start_time)
